In [ ]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# RESOLVER PROBLEMA DE VERSÃO PYTHON X PYSPARK WORKERS

In [ ]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.types import *

import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("teste") \
      .getOrCreate()

In [ ]:
df = spark.read.option("header",True) \
          .csv("relatorio_tratado.csv")
df.printSchema()
df.show()

In [ ]:
# Renomear Colunas
df = df.withColumnRenamed("categorias", "Categoria").withColumnRenamed("categoria_id", "Tipo_categoria").withColumnRenamed("nome2", "Equipamentos") \
.withColumnRenamed("condominio_id", "Tipo_condominio").withColumnRenamed("nome5", "Condominio").withColumnRenamed("data_instalacao", "Instalacao") 

df.show(5)

In [ ]:
# Criar coluna com data atual
df = df.withColumn("Data_atual", current_date())
df.show(5)


In [ ]:
# Ver os tipos de categorias dentro da coluna e ordenar o Tipo_categoria
#df.select("Tipo_categoria").distinct().orderBy("Tipo_categoria").show()

In [ ]:
#df_tratado = df.select("Categoria", "Equipamentos", "Condominio", "testado").orderBy("Categoria").show()
df_nao_testados = df.select("Categoria", "Equipamentos", "Condominio", "testado") \
                   .orderBy("Categoria") \
                   .where(df["testado"] == "NÃO")


df_nao_testados.show()

In [ ]:
# Equipamentos Testados
df_nao_testados = df.select("Categoria", "Equipamentos", "Condominio", "testado") \
                   .orderBy("Categoria") \
                   .where(df["testado"] == "SIM")


df_nao_testados.show(df_nao_testados.count(), truncate=False)

In [ ]:
contagem_testados = df.groupBy("testado").count()

# Mostrar o resultado da contagem
contagem_testados.show()

In [ ]:
# Converter o DataFrame do Spark para o Pandas
pandas_df = contagem_testados.toPandas()


plt.pie(pandas_df['count'], labels=pandas_df['testado'], autopct='%1.1f%%')
plt.axis('equal') 
plt.title('Máquinas Testadas')
plt.show()

In [ ]:
# Contar o número de máquinas em cada condomínio e colocar por odem decrescente
contagem_maquinas_por_condominio = df.groupBy('Condominio').count().withColumnRenamed('count', 'quantidade').orderBy('quantidade', ascending=False)

# Mostrar o resultado da contagem
contagem_maquinas_por_condominio.show(contagem_maquinas_por_condominio.count(), truncate=False)


In [ ]:
# Contar o número de máquinas em cada condomínio
contagem_maquinas_por_condominio = df.groupBy('Condominio').count()

# Ordenar o DataFrame pela contagem em ordem decrescente
contagem_maquinas_ordenado = contagem_maquinas_por_condominio.orderBy('count', ascending=False)

# Coletar o valor máximo da contagem de máquinas
max_qtde_maquinas = contagem_maquinas_ordenado.first()['count']

# Exibir somente os condomínios com a maior quantidade de máquinas
contagem_maquinas_ordenado.filter(col("count") == max_qtde_maquinas).show()



In [ ]:
# Converter o DataFrame do Spark para o Pandas
pandas_contagem_maquinas = contagem_maquinas_por_condominio.toPandas()

# Ordenar o DataFrame por contagem (opcional)
pandas_contagem_maquinas = pandas_contagem_maquinas.sort_values(by='count', ascending=False)

# Plotar o gráfico de barras
plt.figure(figsize=(20, 10))  # Ajustar o tamanho do gráfico (opcional)
plt.bar(pandas_contagem_maquinas['Condominio'], pandas_contagem_maquinas['count'])
plt.xlabel('Condomínio')
plt.ylabel('Número de Máquinas')
plt.title('Número de Máquinas por Condomínio')
plt.xticks(rotation=90)  # Rotacionar os rótulos do eixo x (opcional para legibilidade)
plt.tight_layout()  # Ajustar o layout para evitar cortes
plt.show()


In [ ]:
# Filtrar valores fracionados
pandas_contagem_maquinas = pandas_contagem_maquinas[pandas_contagem_maquinas['count'] > 1]

# Plotar o gráfico de barras horizontal
plt.figure(figsize=(10, 8))
plt.barh(pandas_contagem_maquinas['Condominio'], pandas_contagem_maquinas['count'], color='skyblue')
plt.xlabel('Número de Máquinas')
plt.ylabel('Condomínio')
plt.title('Número de Máquinas por Condomínio')
plt.tight_layout()
plt.show()


In [ ]:
# Plotar o gráfico de barras horizontal
plt.figure(figsize=(10, 8))  # Tamanho da figura (opcional)
plt.barh(pandas_contagem_maquinas['Condominio'], pandas_contagem_maquinas['count'], color='skyblue')
plt.xlabel('Número de Máquinas')
plt.ylabel('Condomínio')
plt.title('Número de Máquinas por Condomínio')
plt.tight_layout()  # Ajuste do layout para evitar cortes (opcional)
plt.show()

